## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables

# Links:

[[citizens_st_mobile]]
[[subscriptions_st_mobile]]
[[entries_installation_points_dir_partner]]
[[installation_point_st_partner]]
[[citizen_payments_st_mobile]]
[[intercoms_st_partner]]

In [7]:
query_text = """--sql
    CREATE TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch 
    (
        `report_date` Date,
        `intercom_uuid` String,
        `motherboard_id` String,
        `city` String,
        `full_address` String,
        `company_name` String,
        `partner_uuid` String,
        `partner_lk` String,
        `tin` String,
        `citizen_id_in_flat_with_subscriptions` UInt32,
        `payments_amount` UInt32,
        `activated_citizen_id` UInt32,
        `subscribed_citizen_id` UInt32,
        `flat_uuid` UInt32
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

___

In [8]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 27 MINUTE TO db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS 
WITH citizens_st_mobile AS(
	SELECT
		report_date,
		citizen_id,
		address_uuid,
		flat_uuid,
		state
	FROM db1.`citizens_st_mobile_ch` 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND `state` = 'activated'
	),
	subscriptions_st_mobile AS(
	SELECT 
		report_date,
		citizen_id,
		state
	FROM db1.subscriptions_st_mobile_ch
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND state = 'activated'
	),
	installation_point_st AS (
	SELECT 
		report_date,
		installation_point_id
	FROM db1.installation_point_st_partner_ch
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
	),
	intercoms_st_partner AS (
	SELECT
		report_date,
		intercom_uuid,
		partner_uuid,
		installation_point_id
	FROM db1.intercoms_st_partner_ch 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
	),
	citizen_payments_st_mobile AS (
	SELECT
		report_date,
		citizen_id,
		amount
	FROM db1.citizen_payments_st_mobile_ch 
	WHERE report_date = dateTrunc('month', report_date) 
		AND report_date ='2025-06-01'
		AND state = 'success'
	),
	t4 AS(
	SELECT  
		intercoms_st_partner.report_date AS report_date,
		city,
		company_name,
		partner_lk,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		intercoms_st_partner.partner_uuid AS partner_uuid,
		motherboard_id,
		full_address,
		tin
	FROM intercoms_st_partner
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN installation_point_st
		ON installation_point_st.`installation_point_id` = intercoms_st_partner.`installation_point_id`
		AND installation_point_st.`report_date` = intercoms_st_partner.`report_date`
	LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points 
			ON installation_point_st.`installation_point_id` = entries_installation_points.`installation_point_id`
	WHERE intercoms_st_partner.report_date  = dateTrunc('month', intercoms_st_partner.report_date ) 
		AND intercoms_st_partner.report_date ='2025-06-01'
	),
	--
	t1 AS (SELECT
		citizens_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		COUNT(DISTINCT if(citizens_st_mobile.citizen_id !=0, citizens_st_mobile.citizen_id ,Null)) as activated_citizen_id,	
		COUNT(DISTINCT if(subscriptions_st_mobile.citizen_id !=0, subscriptions_st_mobile.citizen_id ,Null)) as subscribed_citizen_id,	
		COUNT(DISTINCT if(citizens_st_mobile.flat_uuid !='', citizens_st_mobile.flat_uuid ,Null)) as flat_uuid
	FROM citizens_st_mobile
	LEFT JOIN  subscriptions_st_mobile
			ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
			AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
			AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizens_st_mobile.report_date 
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	GROUP BY report_date,
			intercom_uuid
	),
	--
	t2 AS (SELECT
		citizen_payments_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		sum(amount) AS payments_amount
	FROM citizen_payments_st_mobile
	LEFT JOIN citizens_st_mobile 
			ON citizen_payments_st_mobile.citizen_id = citizens_st_mobile.citizen_id  
			AND citizen_payments_st_mobile.report_date = citizens_st_mobile.report_date 
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id` 
			AND installation_point_st.`report_date` = citizen_payments_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizen_payments_st_mobile.report_date
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	GROUP BY 	
			report_date,
			intercom_uuid
	),
	--
	t3 AS (SELECT
		report_date,
		intercom_uuid,
		count(if(citizen_id !=0,citizen_id,NULL)) AS citizen_id_in_flat_with_subscriptions
	FROM
		(SELECT
			installation_point_st.report_date AS report_date,
			installation_point_st.citizen_id AS citizen_id,
			installation_point_st.installation_point_id AS installation_point_id,
			installation_point_st.flat_with_sub_active AS flat_with_sub_active,
			intercom_uuid
		FROM
			(SELECT 
				citizens_st_mobile.report_date AS report_date,
				citizens_st_mobile.citizen_id AS citizen_id,
				installation_point_st.installation_point_id AS installation_point_id,
				if(subscriptions_st_mobile.state = 'activated',1,0) AS if_sub_active,
				max(if_sub_active) OVER (partition by citizens_st_mobile.flat_uuid, citizens_st_mobile.report_date ORDER BY citizens_st_mobile.report_date DESC) AS flat_with_sub_active
			FROM citizens_st_mobile AS citizens_st_mobile
			LEFT JOIN subscriptions_st_mobile 
				ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
				AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
			LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
				ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
			LEFT JOIN db1.`installation_point_st_partner_ch` AS  installation_point_st
				ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
				AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`) AS installation_point_st
			LEFT JOIN intercoms_st_partner 
				ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
				AND intercoms_st_partner.report_date = installation_point_st.report_date
		WHERE flat_with_sub_active = 1
		)
	GROUP BY report_date,
			intercom_uuid
	)
	--
	SELECT
		t1.report_date AS report_date,
		t1.intercom_uuid AS intercom_uuid,
		t4.full_address AS full_address,
		t4.company_name AS company_name,
		t4.city AS city,
		t4.partner_uuid AS partner_uuid,
		t4.partner_lk AS partner_lk,
		t4.motherboard_id AS motherboard_id,
		t4.tin AS tin,
		payments_amount,
		activated_citizen_id,
		subscribed_citizen_id,
		flat_uuid,
		citizen_id_in_flat_with_subscriptions
	FROM  t1 
	LEFT JOIN t2 ON t1.report_date = t2.report_date
				AND t1.intercom_uuid = t2.intercom_uuid
	LEFT JOIN t3 ON t1.report_date = t3.report_date
				AND t1.intercom_uuid = t3.intercom_uuid
	LEFT JOIN t4  ON t1.report_date = t4.report_date
				AND t1.intercom_uuid = t4.intercom_uuid
	"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [12]:
query_text = """
    SELECT
        *
    FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    WHERE intercom_uuid !=''
        AND full_address !=''
    ORDER BY report_date DESC
    LIMIT 10

    """
ch.query_run(query_text)

report_date,intercom_uuid,motherboard_id,city,full_address,company_name,partner_uuid,partner_lk,tin,citizen_id_in_flat_with_subscriptions,payments_amount,activated_citizen_id,subscribed_citizen_id,flat_uuid
date,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64
2025-06-01,"""b65382f6-4059-425c-b721-0872a2…","""2236149375""","""Киров""","""Россия, Кировская область, мун…","""ООО Добрыня ПРО""","""1c64b848-e82c-46ee-9d0d-c74093…","""121833""","""4345434743""",5,0,61,3,40
2025-06-01,"""ebb7a1c6-5e44-4426-8244-d6a3b9…","""2121121491""","""Талдыкорган""","""Казахстан, Алматинская область…","""Метаком Сервис""","""c2834e56-9f4f-454a-bb29-102645…","""123989""","""""",0,0,117,0,59
2025-06-01,"""8970468b-fdd0-4d24-90a9-85aaf5…","""2125129517""","""Омск""","""Россия, Омская область, городс…","""Рестро""","""d0985c07-4e70-40a0-854a-fac9af…","""151668""","""""",9,0,26,4,15
2025-06-01,"""4af20e38-ce17-4641-9371-525ecd…","""2121121263""","""Озерск""","""Россия, Челябинская область, О…","""ООО Bид Плюс ПРО""","""963e8bcc-c6c3-46ee-958f-412c22…","""120696""","""7422048677""",3,0,26,2,15
2025-06-01,"""bb995a0f-9b55-476d-b54a-75d242…","""2436158914""","""Истра""","""Россия, Московская область, го…","""Кузнецов Андрей Александрович …","""ab58f21a-8dbe-4188-baf7-9dc694…","""120467""","""501700274500""",8,0,15,4,7
2025-06-01,"""2e583cc8-c12c-4575-a171-4327df…","""2236149147""","""Нижний Новгород""","""Россия, Нижегородская область,…","""ИП Красильников Андрей Владими…","""490ea93f-b1ad-480b-a280-7c9987…","""121988""","""526300547480""",18,0,56,8,31
2025-06-01,"""2b26b54a-fb30-44ba-99fd-b337b3…","""2235147042""","""Вологда""","""Россия, Вологодская область, г…","""ООО Сокол ТВ ПРО""","""23dcf97a-72de-4709-aeb0-e62082…","""124775""","""3527010320""",0,0,9,0,5
2025-06-01,"""ddb58084-9294-480b-ada2-b8b9a1…","""2235141848""","""Агрыз""","""Россия, Республика Татарстан, …","""ООО ТВК-СЕРВИС ПРО""","""d2fbd121-9d7d-4896-aeda-e7a85d…","""126800""","""1601006591""",12,0,23,5,9
2025-06-01,"""5c60fc93-32fa-4c3c-acc2-8dff49…","""2148129819""","""Новокузнецк""","""Россия, Кемеровская область, Н…","""Цифрал Сервис УК""","""0e7236ac-b8bd-4ab0-8634-d165ad…","""120345""","""""",0,0,28,0,13


In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,1313005,942ed803-976d-4ff9-ad7e-e1a5a1b371f7


In [6]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
70896080,"""25f440bd-3ba8-4b01-8f6c-31227c…"
